### 训练数据缩减

In [1]:
import numpy as np
import os
import pandas as pd
import pickle as pkl

data_path = os.path.join("data", "dataset2train_v2.pkl")
with open(data_path, "rb") as f:
    check_data = pkl.load(f)

print("Sample Data:")
print(check_data[0][0][0])

Sample Data:
('uid_118412', array([ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        , -0.00130648, -0.01633448]), array([0., 1., 0., 0., 0., 0., 0.]), datetime.date(2022, 8, 1), 1, False, False, (4, 0.3261117316892494))


In [19]:
mon_string = ["Aug", "Sep"]
total_adv_order_num = 0
total_order_num = 0

for mon in range(len(check_data)):
    print("{}:".format(mon_string[mon]))
    num_adv_orders = 0
    num_total = 0
    num_redelv = 0
    num_adv_redelv = 0
    level_dist = {}
    adv_level_dist = {}
    for day in range(len(check_data[mon])):
        for i in range(len(check_data[mon][day])):
            num_total += 1
            if check_data[mon][day][i][-3]:
                num_adv_orders += 1
                if check_data[mon][day][i][-2]:
                    num_adv_redelv += 1

                if check_data[mon][day][i][-1][0] not in adv_level_dist:
                    adv_level_dist[check_data[mon][day][i][-1][0]] = 1
                else:
                    adv_level_dist[check_data[mon][day][i][-1][0]] += 1

            elif check_data[mon][day][i][-2]:
                num_redelv += 1

            if check_data[mon][day][i][-1][0] not in level_dist:
                level_dist[check_data[mon][day][i][-1][0]] = 1
            else:
                level_dist[check_data[mon][day][i][-1][0]] += 1
    total_adv_order_num += num_adv_orders
    total_order_num += num_total
    
    print("ADV order_num: {} / total num: {} = {}".format(num_adv_orders, num_total, num_adv_orders/num_total))
    print("redelv num: {} / total num: {} = {}".format(num_redelv, num_total, 1-(num_redelv/num_total)))
    print("ADV redelv num: {} / ADV order_num: {} = {}".format(num_adv_redelv, num_adv_orders, 1-(num_adv_redelv/num_adv_orders)))
    
    np.set_printoptions(precision=4)
    level_cnt = [level_dist[i] for i in sorted(level_dist)]
    level_hist = np.array([i/np.sum(level_cnt) for i in level_cnt])
    print("total level_dist: {}, hist: {}".format(level_cnt, level_hist))
    adv_level_cnt = [adv_level_dist[i] for i in sorted(adv_level_dist)]
    adv_level_hist = np.array([i/np.sum(adv_level_cnt) for i in adv_level_cnt])

    print("ADV level_dist:      {},         hist: {}".format(adv_level_cnt, adv_level_hist))
    print()
print("total order_num: {}".format(total_order_num))
print("total ADV order_num: {}".format(total_adv_order_num))

Aug:
ADV order_num: 1194 / total num: 29625 = 0.04030379746835443
redelv num: 1681 / total num: 29625 = 0.9432573839662447
ADV redelv num: 270 / ADV order_num: 1194 = 0.7738693467336684
total level_dist: [3000, 4500, 8385, 10473, 3267], hist: [0.1013 0.1519 0.283  0.3535 0.1103]
ADV level_dist:      [28, 112, 504, 429, 121],         hist: [0.0235 0.0938 0.4221 0.3593 0.1013]

Sep:
ADV order_num: 707 / total num: 35174 = 0.02010007391823506
redelv num: 2042 / total num: 35174 = 0.9419457553875021
ADV redelv num: 158 / ADV order_num: 707 = 0.7765205091937766
total level_dist: [3000, 4500, 10747, 12631, 4296], hist: [0.0853 0.1279 0.3055 0.3591 0.1221]
ADV level_dist:      [7, 39, 286, 295, 80],         hist: [0.0099 0.0552 0.4045 0.4173 0.1132]

total order_num: 64799
total ADV order_num: 1901


### 继续筛选无人车订单

In [2]:
new_train_data = []
for mon_data in check_data:
    new_mon_data = []
    for day_data in mon_data:
        new_day_data = [i for i in day_data if i[-3]]
        new_mon_data.append(new_day_data)
    new_train_data.append(new_mon_data)

In [11]:
print(len(new_train_data))
print(len(new_train_data[0]))
print(len(new_train_data[0][0]))
print(len(new_train_data[1]))
print(len(new_train_data[1][0]))

for i in new_train_data[0][0]:
    print(i[-1])

2
30
70
30
49
(1, 0.32611173168924956)
(4, 0.3261117316892494)
(1, 0.740473603892424)
(1, 0.740473603892424)
(2, 0.7964934609959796)
(3, 1.3402043200512643)
(3, 1.3402043200512646)
(4, 0.7964934609959796)
(4, 0.7964934609959796)
(1, 1.6407512468739265)
(4, 0.3261117316892494)
(3, 1.337104202693495)
(3, 1.3402043200512646)
(4, 0.3261117316892494)
(4, 0.7964934609959795)
(3, 1.3402043200512643)
(1, 0.740473603892424)
(4, 0.8811230744056688)
(1, 0.7964934609959796)
(1, 0.7964934609959796)
(3, 0.881123074405669)
(2, 1.3402043200512648)
(4, 0.3261117316892494)
(5, 0.043546946208700185)
(2, 0.7964934609959796)
(3, 1.219656760841688)
(4, 0.3261117316892494)
(4, 0.7964934609959796)
(4, 0.7964934609959796)
(4, 0.37349918295764173)
(4, 0.7964934609959795)
(3, 0.881123074405669)
(2, 1.3490864960246274)
(3, 1.219656760841688)
(3, 0.881123074405669)
(5, 0.12497481033076557)
(4, 0.3734991829576419)
(3, 1.3490864960246274)
(4, 0.3261117316892494)
(2, 1.3490864960246274)
(5, 0.043546946208700185)
(3, 

In [12]:
with open(os.path.join("data", "dataset2train_v3.pkl"), "wb") as f:
    pkl.dump(new_train_data, f)

### 预测新数据集准确率

In [3]:
import numpy as np
import os
import pandas as pd
import pickle as pkl
with open(os.path.join("data", "dataset2train_v3.pkl"), "rb") as f:
    new_train_data = pkl.load(f)

with open(os.path.join("data", "dataset2train_v2.pkl"), "rb") as f:
    org_train_data = pkl.load(f)

with open(os.path.join("data", "userdata2train_v2.pkl"), "rb") as f:
    user_data = pkl.load(f)

In [7]:
from policy_model import CDPModel_DQN_with_pretrain, CDPModel_TDQN_with_pretrain
from env import OrderEnv2
from order_dataset import OrderDataset
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from env import ACTION_SPACE, OBSERVATION_SPACE_LITE
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

timestp = int(time.time())
model_name = "TDQN_ptv808_test"
dir_path = "weights/pretrain/{}_{}".format(model_name, timestp)
file_name = "state_dict.pt"
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
        
_device = torch.device("cuda:0")
# env = OrderEnv2()

model = CDPModel_TDQN_with_pretrain(obs_space=ACTION_SPACE,
                                       action_space=OBSERVATION_SPACE_LITE,
                                       num_outputs=256,
                                       model_config={},
                                       name="CDP_TDQN_PRETRAIN",
                                       pre_train=True, device=_device, recurrent=False
                                       ).to(_device)
                                

clsf = nn.Linear(256, 7).to(_device)
level_est = nn.Linear(256, 5).to(_device)


xent_loss_fn = nn.CrossEntropyLoss()

    # optim = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
optim = torch.optim.Adam(model.parameters(), lr=2e-5)
lr_sched = StepLR(optim, step_size=50, gamma=0.5)
train_dataset = OrderDataset(0, split="train", max_length=20000, data=org_train_data, user_data=user_data)
data_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
eval_data_loader = DataLoader(OrderDataset(0, max_length=2000, user_list=train_dataset.get_user_list(), split="test", data=org_train_data, user_data=user_data), batch_size=128, shuffle=True)

max_acc = -np.inf
for i in range(50):
    avg_loss = 0
    avg_acc = 0
    avg_lvl_acc = 0
    model.train()
    for x, y1, y2 in tqdm(data_loader):
        x = [t.to(_device) for t in x]
        y1 = y1.to(_device)
        y2 = y2.to(_device)
        input_dict = {"obs": [x]}
        feature, _ = model(input_dict)
        logits = clsf(feature)
        lvl_pred = level_est(feature)
        cls_loss = xent_loss_fn(logits, y1)
        # cls_loss = 0
        lvl_loss = xent_loss_fn(lvl_pred, y2)
        # lvl_loss = 0
        loss = cls_loss + lvl_loss
        optim.zero_grad()
        loss.backward()
        optim.step()
        avg_loss = avg_loss + loss.detach().item()
        avg_acc  = avg_acc + torch.mean(torch.eq(torch.argmax(logits, dim=-1), torch.argmax(y1, dim=-1)).float()).detach().item()
        avg_lvl_acc  = avg_lvl_acc + torch.mean(torch.eq(torch.argmax(lvl_pred, dim=-1), torch.argmax(y2, dim=-1)).float()).detach().item()
    lr_sched.step()
    print("EP{}: loss: {:.5f}, acc: {:.3f}, lvl_acc: {:.3f}".format(i, avg_loss/len(data_loader), avg_acc/len(data_loader), avg_lvl_acc/len(data_loader)))

    model.eval()
    avg_eval_loss = 0
    avg_eval_acc = 0
    avg_eval_lvl_acc = 0
    avg_eval_hard_acc = 0
    avg_eval_easy_acc = 0
    for eval_x, eval_y1, eval_y2 in eval_data_loader:
        eval_x = [t.to(_device) for t in eval_x]
        eval_y1 = eval_y1.to(_device)
        eval_y2 = eval_y2.to(_device)
        input_dict = {"obs": [eval_x]}
        feature, _ = model(input_dict)
        logits = clsf(feature)
        lvl_pred = level_est(feature)
        cls_loss = xent_loss_fn(logits, eval_y1)
        # cls_loss = 0
        lvl_loss = xent_loss_fn(lvl_pred, eval_y2)
        # lvl_loss = 0
        loss = cls_loss + lvl_loss

        acc = torch.mean(torch.eq(torch.argmax(logits, dim=-1), torch.argmax(eval_y1, dim=-1)).float()).detach().item()
        lvl_acc = torch.mean(torch.eq(torch.argmax(lvl_pred, dim=-1), torch.argmax(eval_y2, dim=-1)).float()).detach().item()
        

        # hard_ind = [i for i, v in enumerate(torch.argmax(eval_y1, dim=-1).detach().cpu().numpy()) if v == 7]
        # hard_logits = torch.argmax(logits, dim=-1).detach().cpu().numpy()
        # hard_logits = np.array([hard_logits[i] for i in hard_ind])
        # hard_acc = sum(hard_logits == 7)/len(hard_ind)

        # easy_ind = [i for i, v in enumerate(torch.argmax(eval_y1, dim=-1).detach().cpu().numpy()) if v != 7] 
        # easy_logits = torch.argmax(logits, dim=-1).detach().cpu().numpy()
        # easy_logits = np.array([easy_logits[i] for i in easy_ind])
        # easy_label = torch.argmax(eval_y1, dim=-1).detach().cpu().numpy()
        # easy_label = np.array([easy_label[i] for i in easy_ind])
        # easy_acc = sum(easy_logits == easy_label)/len(easy_ind)

        avg_eval_loss = avg_eval_loss + loss
        avg_eval_acc = avg_eval_acc + acc
        avg_eval_lvl_acc = avg_eval_lvl_acc + lvl_acc
        # avg_eval_hard_acc = avg_eval_hard_acc + hard_acc
        # avg_eval_easy_acc = avg_eval_easy_acc + easy_acc
    print("EP{} (VAL): loss: {:.5f}, acc: {:.5f}, lvl_acc: {:.5f}".format(i, avg_eval_loss/len(eval_data_loader), avg_eval_acc/len(eval_data_loader), avg_eval_lvl_acc/len(eval_data_loader)))
    if avg_eval_acc/len(eval_data_loader) > max_acc:
        torch.save(model.state_dict(), os.path.join(dir_path, file_name))
        torch.save(clsf.state_dict(), os.path.join(dir_path, "clsf.pt"))
        torch.save(level_est.state_dict(), os.path.join(dir_path, "lest.pt"))
        print("New Model Saved.")
        max_acc = avg_eval_acc/len(eval_data_loader)
    
# torch.save(model.state_dict(), os.path.join(dir_path, file_name))
# torch.save(clsf.state_dict(), os.path.join(dir_path, "clsf.pt"))
# torch.save(level_est.state_dict(), os.path.join(dir_path, "lest.pt"))

100%|██████████| 157/157 [00:07<00:00, 19.88it/s]


EP0: loss: 3.68823, acc: 0.158, lvl_acc: 0.178
EP0 (VAL): loss: 3.53499, acc: 0.27031, lvl_acc: 0.17686
New Model Saved.


100%|██████████| 157/157 [00:07<00:00, 21.60it/s]


EP1: loss: 3.56317, acc: 0.272, lvl_acc: 0.189
EP1 (VAL): loss: 3.17239, acc: 0.52510, lvl_acc: 0.19648
New Model Saved.


100%|██████████| 157/157 [00:07<00:00, 20.03it/s]


...

EP47: loss: 1.71303, acc: 0.714, lvl_acc: 0.697
EP47 (VAL): loss: 1.76702, acc: 0.67344, lvl_acc: 0.67744


100%|██████████| 157/157 [00:07<00:00, 21.72it/s]


EP48: loss: 1.69301, acc: 0.719, lvl_acc: 0.701
EP48 (VAL): loss: 1.75720, acc: 0.68213, lvl_acc: 0.67607


100%|██████████| 157/157 [00:07<00:00, 21.65it/s]


EP49: loss: 1.69103, acc: 0.718, lvl_acc: 0.698
EP49 (VAL): loss: 1.72829, acc: 0.69277, lvl_acc: 0.67783


### 加载模型测试新数据

In [8]:
state_dict = torch.load(os.path.join(dir_path, file_name))
model.load_state_dict(state_dict, strict=False)

state_dict = torch.load(os.path.join(dir_path, "clsf.pt"))
clsf.load_state_dict(state_dict, strict=False)

state_dict = torch.load(os.path.join(dir_path, "lest.pt"))
level_est.load_state_dict(state_dict, strict=False)

<All keys matched successfully>

In [39]:
for i in range(2):
    test_dataset = OrderDataset(i, split="all", max_length=20000, data=new_train_data, user_data=user_data)
    data_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

    model.eval()
    avg_eval_loss = 0
    avg_eval_acc = 0
    avg_eval_lvl_acc = 0
    avg_eval_hard_acc = 0
    avg_eval_easy_acc = 0
    for eval_x, eval_y1, eval_y2 in data_loader:
        eval_x = [t.to(_device) for t in eval_x]
        eval_y1 = eval_y1.to(_device)
        eval_y2 = eval_y2.to(_device)
        input_dict = {"obs": [eval_x]}
        feature, _ = model(input_dict)
        logits = clsf(feature)
        lvl_pred = level_est(feature)
        cls_loss = xent_loss_fn(logits, eval_y1)
        # cls_loss = 0
        lvl_loss = xent_loss_fn(lvl_pred, eval_y2)
        # lvl_loss = 0
        loss = cls_loss + lvl_loss

        acc = torch.mean(torch.eq(torch.argmax(logits, dim=-1), torch.argmax(eval_y1, dim=-1)).float()).detach().item()
        lvl_acc = torch.mean(torch.eq(torch.argmax(lvl_pred, dim=-1), torch.argmax(eval_y2, dim=-1)).float()).detach().item()
        

        # hard_ind = [i for i, v in enumerate(torch.argmax(eval_y1, dim=-1).detach().cpu().numpy()) if v == 7]
        # hard_logits = torch.argmax(logits, dim=-1).detach().cpu().numpy()
        # hard_logits = np.array([hard_logits[i] for i in hard_ind])
        # hard_acc = sum(hard_logits == 7)/len(hard_ind)

        # easy_ind = [i for i, v in enumerate(torch.argmax(eval_y1, dim=-1).detach().cpu().numpy()) if v != 7] 
        # easy_logits = torch.argmax(logits, dim=-1).detach().cpu().numpy()
        # easy_logits = np.array([easy_logits[i] for i in easy_ind])
        # easy_label = torch.argmax(eval_y1, dim=-1).detach().cpu().numpy()
        # easy_label = np.array([easy_label[i] for i in easy_ind])
        # easy_acc = sum(easy_logits == easy_label)/len(easy_ind)

        avg_eval_loss = avg_eval_loss + loss
        avg_eval_acc = avg_eval_acc + acc
        avg_eval_lvl_acc = avg_eval_lvl_acc + lvl_acc
        # avg_eval_hard_acc = avg_eval_hard_acc + hard_acc
        # avg_eval_easy_acc = avg_eval_easy_acc + easy_acc
    print("DATASET{} (VAL): loss: {:.5f}, acc: {:.5f}, lvl_acc: {:.5f}".format(i, avg_eval_loss/len(data_loader), avg_eval_acc/len(data_loader), avg_eval_lvl_acc/len(data_loader)))


DATASET0 (VAL): loss: 1.69473, acc: 0.73478, lvl_acc: 0.64475
DATASET1 (VAL): loss: 1.69707, acc: 0.71220, lvl_acc: 0.65679


### 预测新数据集送达时间

In [40]:
all_pred_labels = []
for i in range(2):
    pred_labels = []
    test_dataset = OrderDataset(i, split="all", max_length=200000, data=new_train_data, user_data=user_data)
    data_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
    model.eval()
    avg_eval_loss = 0
    avg_eval_acc = 0
    avg_eval_lvl_acc = 0
    avg_eval_hard_acc = 0
    avg_eval_easy_acc = 0
    for eval_x, eval_y1, eval_y2 in data_loader:
        eval_x = [t.to(_device) for t in eval_x]
        eval_y1 = eval_y1.to(_device)
        eval_y2 = eval_y2.to(_device)
        input_dict = {"obs": [eval_x]}
        feature, _ = model(input_dict)
        logits = clsf(feature)
        labels = torch.argmax(logits, dim=-1).cpu().numpy()
        
        pred_labels.extend(list(labels))
    all_pred_labels.append(pred_labels)

In [44]:
len(all_pred_labels[0])

1194

In [42]:
len(all_pred_labels[1])

707

In [47]:
# 写入预测标签
p_label = 0
prepared_pred_labels = all_pred_labels[0] + all_pred_labels[1]
for mon_data in new_train_data:
    for day_data in mon_data:
        for i, item in enumerate(day_data):
            day_data[i] = item + (prepared_pred_labels[p_label],)
            p_label += 1
assert p_label == len(all_pred_labels[0]) + len(all_pred_labels[1])


In [48]:
# 检查写入情况
new_train_data[0][0][0]

('uid_015497',
 array([ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        , -0.00227784, -0.01302929]),
 array([0., 0., 1., 0., 0., 0., 0.]),
 datetime.date(2022, 8, 1),
 2,
 True,
 False,
 (1, 0.32611173168924956),
 5)

In [52]:
len(new_train_data[0][0][0][1])

29

In [49]:
# 计算准确率
n_correct = 0
for mon_data in new_train_data:
    for day_data in mon_data:
        for i, item in enumerate(day_data):
            if np.argmax(item[2]) == item[-1]:
                n_correct += 1
n_correct / (len(all_pred_labels[0]) + len(all_pred_labels[1]))

0.7264597580220936

In [51]:
# 覆盖新数据集
with open(os.path.join("data", "dataset2train_v3.pkl"), "wb") as f:
    pkl.dump(new_train_data, f)

### 存储user embedding

In [6]:
from env import ACTION_SPACE, OBSERVATION_SPACE_LITE
from policy_model import CDPModel_TDQN_with_pretrain
import torch
model = CDPModel_TDQN_with_pretrain(obs_space=ACTION_SPACE,
                                       action_space=OBSERVATION_SPACE_LITE,
                                       num_outputs=256,
                                       model_config={},
                                       name="CDP_TDQN_PRETRAIN",
                                       pre_train=True, device="cpu", recurrent=False
                                       )

In [7]:
state_dict = torch.load("weights/pretrain/TDQN_ptv808_test_1691451460/state_dict.pt")
model.load_state_dict(state_dict, strict=False)

<All keys matched successfully>

In [9]:

import os 
torch.save(model.user_embedding.state_dict(), os.path.join("weights/pretrain/TDQN_ptv808_test_1691451460", "user_embedding.pt"))